In [1]:
import torch
import torch.nn as nn

In [2]:
class VGG16(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG16, self).__init__()
        self.in_channels = in_channels
        
        self.layer1 = self._make_layer(size=3)
        self.conv = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=2, bias=False)
        self.bn = nn.BatchNorm1d(num_features=128)
        
        
        self.fc1 = nn.Linear(in_features=128, out_features=128*2, bias=False)
        self.fc2 = nn.Linear(in_features=128*2, out_features=num_classes)
        
        self.relu = nn.ReLU(inplace=True)
        self.bn1 = nn.BatchNorm1d(num_features=6)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn(self.conv(x))
        
        x = x.reshape(-1, 6, 128)
        
        x = self.bn1(self.fc1(x))
        x = self.fc2(self.relu(x))
        return x
    
    def _make_layer(self, size=None):
        layers = []
        out_channels = 512
        
        if size == 3:
            for j in range(size-1):
                for i in range(size):
                    layer = nn.Sequential(
                            nn.Conv1d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3, bias=False),
                            nn.BatchNorm1d(num_features=out_channels),
                            nn.ReLU(inplace=True))
                    layers.append(layer)
                    self.in_channels = out_channels
                layers.append(nn.Sequential(nn.MaxPool1d(kernel_size=2)))
            out_channels = out_channels // 2 # 512 ---> 256 
            
            for j in range(size-1):
                layer = nn.Sequential(
                            nn.Conv1d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3, bias=False),
                            nn.BatchNorm1d(num_features=out_channels), 
                            nn.ReLU(inplace=True))
                layers.append(layer)
                self.in_channels = out_channels
            
            out_channels = out_channels // 2 # 256 ---> 128
            layer = nn.Sequential(
                            nn.Conv1d(in_channels=self.in_channels, out_channels=out_channels, kernel_size=3, bias=False),
                            nn.BatchNorm1d(num_features=out_channels), 
                            nn.ReLU(inplace=True),
                            nn.MaxPool1d(kernel_size=2))
            layers.append(layer)
            self.in_channels = out_channels
                
        return nn.Sequential(*layers)

In [3]:
model = VGG16(in_channels=6044, num_classes=9)

x = torch.randn((1, 6044, 99))
model(x).shape

torch.Size([1, 6, 9])